In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.dpi'] = 70 #display 70 dpi in Jupyter Notebook, may consider100 dpi 
plt.rcParams['savefig.dpi'] = 300 #define 300 dpi for saving figures

import seaborn as sns
## here are some settings 
sns.set_style('whitegrid')
sns.set(rc={"figure.dpi":70, 'savefig.dpi':300}) #defining dpi setting
sns.set_context('notebook')
sns.set_style("ticks")
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

C:\Users\LMS-Sapkals\AppData\Local\Temp\ipykernel_2384\4110068169.py:13: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('retina')


In [2]:
# Tells matplotlib to display images inline instead of a new window
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from time import time
import timeit #imports timeit module
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV
start_time = timeit.default_timer() #defines start time so computational time can be calculated
print("Hello World")
elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

Hello World
---Run time is 0.00012199999764561653 seconds ---



In [3]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.preprocessing import PolynomialFeatures

import statistics
import sklearn.linear_model as skl_lm
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn import neighbors
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.tree import DecisionTreeRegressor 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.tree import export_graphviz
from sklearn.svm import SVC

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import mean_squared_error

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from tqdm import tqdm
import wfdb
import numpy as np
import pandas as pd

In [4]:
# import os
# cwd = os.getcwd()
# print(cwd)
# os.chdir('/Users/chenc/000---STT450-Jupyter Notebook/000--STT450-550/') 
#data = load_breast_cancer(as_frame = True); 
#df = data.frame; 
#print(df.shape)
# In this block use the code we looked at on 6/30 as a reference to save our rr-intervals as a .
rlist = []
records = 'mit-bih-raw\RECORDS' # Replace the _______ with the name of the records file in your MIT-BIH data folder
with open(records) as rfile: #Then we open the file 
                             #The 'with' command only opens the file while we are in it. Automatically closes the file when we're not
    for record in rfile:  # Then we iterate through the lines in the file
        record = record[0:len(record)-1] # Remove any erronious new line characters at the end ('\n')
        rlist.append(record) # Then build an array with it
        

###### Step 1: Initialize all Arrays
             # Below, replace all of the ___ with the command that declares an array/list
             # hint: https://stackoverflow.com/questions/1514553/how-to-declare-an-array-in-python
samples = [] # will house the samples of all subjects
good_list = [] # will list the names of the subjects we successfully extracted
bad_list = [] # will house the names of the subjects we failed to extract
qrs = [] # will house the indices of R-Peaks for all subjects
atr_label = [] # will house the labels for each rhythm annotation for all subjects
atr_locs = [] # will house the locations corresponding to the rhythm annotation labels


###### Step 2: Extract Information
for x in tqdm(rlist): #this will iterate through te records that we found above
  
    try: # A try statement will run the except statement if for some reason the try commands fail
         # In this case I use the try statement because one of the subjects has no signal data causing failure
         # I then use bad_list and good_list so that all of the indices in rlist match with the arrays we initialized in Step 1, above
        ######################################################
            # Below find the wfdb function that will return the information that is described below 
        # Then replace _____ with the correct function call
        samp = wfdb.rdsamp('mit-bih-raw/'+x) # wfdb._____(file_location) will read the signal & header data and return a 2 value array
            # samp[0] - the signal data is the raw reading from the ecg. Each value is a sample taken.
            # samp[1] - the header data includes things about the signal data such as:
              # samples per section, denoted 'fs'
              # number of signals, denoted 'n_sig'
            
        ######################################################
        samples.append(samp) #add it to our array for all subject
            #What is our file extension that has the annotation we want? Find it here and replace _____ with it 
            #hint: READ THE VARIABLE NAMES!!!!
        qrs_tmp = wfdb.rdann('mit-bih-raw/'+x, extension='qrs') #extract the QRS Info
        qrs_locs = np.array(qrs_tmp.sample, dtype='int') #Get just the loccation of R-Peaks from the QRS Info
        qrs.append(qrs_locs) # Add to our array for all subjects
        
            #Do the same thing here
        atr = wfdb.rdann('mit-bih-raw/'+x,extension='atr') #extract the atr info which stores the rhythm type(s) over the whole signal
        atr_label.append(atr.aux_note) # aux_note stores the type of rhythm - main two are '(N' for normal and '(AFIB' for AFIB
        atr_locs.append(np.append(atr.sample, len(samp[0]))) #I add the length of the whole sample to the end for better visualization later
        
        good_list.append(x) # when all extraction is successful append the record name to good_list
    except Exception as exep:
        print(exep) # Alert the user of an exception
        bad_list.append(x) # add to the bad list

        

  0%|          | 0/25 [00:00<?, ?it/s]

sampto must be greater than sampfrom
sampto must be greater than sampfrom


100%|██████████| 25/25 [00:15<00:00,  1.57it/s]


In [5]:
# df3 = pd.read_csv('Data/Carseats.csv')
# #df3.info()
# print(); df3.head(5)
features_df_list = []
for subject in good_list:
    data = pd.read_csv(f'subject-features-dataframes/subject{subject}.csv')
    data = data.assign(Subject = subject)
    #print(data)
    features_df_list.append(data)
features_maindf = pd.concat(features_df_list, ignore_index = True)

In [6]:
# df3['High'] = df3.Sales.map(lambda x: 1 if x>8 else 0)
# df3.ShelveLoc = pd.factorize(df3.ShelveLoc)[0]
# df3.Urban = df3.Urban.map({'No':0, 'Yes':1})
# df3.US = df3.US.map({'No':0, 'Yes':1})
# #df3.info()
# print(); df3.head(5)
#X = df3.drop(['Sales', 'High'], axis = 1)
#y = df3.High
##print(X.shape)
#print(y.shape)

In [7]:
x_values = ['StoS', 'StoR', 'StoL', 'RtoS', 'RtoR', 'RtoL', 'LtoS',
       'LtoR', 'LtoL', 'STD', 'CoefVar', 'Range', 'Volatility', 'ShEn',
       'AppEn', 'SampEn']
y_values = ['Rhythm']
features_df_list[0]

,Unnamed: 0,StoS,StoR,StoL,RtoS,RtoR,RtoL,LtoS,LtoR,LtoL,STD,CoefVar,Range,Volatility,ShEn,AppEn,SampEn,Rhythm,Subject
0,0,0.250000,0.125000,0.083333,0.083333,0.208333,0.083333,0.083333,0.041667,0.041667,0.180045,0.259401,0.620,0.024543,1.050913,0.807078,1.004927,0,04015
1,1,0.208333,0.166667,0.041667,0.166667,0.291667,0.041667,0.041667,0.041667,0.000000,0.181063,0.314259,0.584,0.034714,0.915595,0.722627,0.867501,0,04015
2,2,0.083333,0.083333,0.166667,0.041667,0.375000,0.041667,0.166667,0.041667,0.000000,0.176833,0.224044,0.620,0.033321,1.038812,0.780945,0.871395,0,04015
3,3,0.166667,0.125000,0.000000,0.083333,0.416667,0.083333,0.041667,0.041667,0.041667,0.166312,0.240948,0.544,0.019996,0.917357,0.551398,0.559616,0,04015
4,4,0.291667,0.208333,0.041667,0.125000,0.000000,0.125000,0.166667,0.000000,0.041667,0.190016,0.289659,0.552,0.033423,0.989094,0.638499,0.850539,0,04015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1743,1743,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.011808,0.014536,0.064,0.000326,0.000000,0.000000,0.083382,0,04015
1744,1744,0.000000,0.041667,0.000000,0.000000,0.958333,0.000000,0.000000,0.000000,0.000000,0.020241,0.024859,0.088,0.000604,0.167944,0.000000,0.083382,0,04015
1745,1745,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.016860,0.020821,0.076,0.000486,0.000000,0.000000,0.083382,0,04015
1746,1746,0.000000,0.041667,0.000000,0.000000,0.958333,0.000000,0.000000,0.000000,0.000000,0.024815,0.030645,0.080,0.000595,0.167944,0.000000,0.083382,0,04015


### An AdaBoost classifier.

The argument `n_estimators = 500` indicates that we want 500 trees

class sklearn.ensemble.AdaBoostClassifier(base_estimator=None, *, n_estimators=50, learning_rate=1.0, algorithm='SAMME.R', random_state=None)

In [8]:
crossvalidation = KFold(n_splits=23)
#for df in features_df_list:
df = features_maindf
acc_score = [];
Truth = [];
Output = [];
n_estimators_list = []
#max_depth_list = []
for idx, subject in tqdm(enumerate(good_list)):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values].values.ravel(), df.loc[test_index,y_values]

    temp_model = AdaBoostClassifier(n_estimators=500, learning_rate = 0.1, algorithm="SAMME.R", random_state=2)
    param_distributions = {'n_estimators': randint(475, 526)}
    search = RandomizedSearchCV(estimator=temp_model, param_distributions=param_distributions, random_state=0)
    search.fit(X_train, y_train)

    n_estimators_list.append(search.best_params_['n_estimators'])
    #max_depth_list.append(search.best_params_['max_depth'])

    model = AdaBoostClassifier(n_estimators=search.best_params_['n_estimators'], 
                               learning_rate=0.1,algorithm="SAMME.R", random_state=2)
    model.fit(X_train, y_train)    

    #model = AdaBoostClassifier(n_estimators=500, learning_rate = 0.1, algorithm="SAMME.R", random_state=2)
    #model.fit(X_train,y_train)

    pred_values = model.predict(X_test)
    pred_prob = model.predict_proba(X_test)[:, 1]
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list
elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print(acc_score)
print(confusion_matrix(Truth, Output))
print(classification_report(Truth, Output))
print(n_estimators_list)
temp_list = []
temp_idx = []
n_estimators_best = []
max_depth_best = []
for acc in acc_score:
    if acc >= 0.95:
        temp_list.append(acc)
        temp_idx.append(acc_score.index(acc))
for idx in temp_idx:
    n_estimators_best.append(n_estimators_list[idx])
    #max_depth_best.append(max_depth_list[idx])

23it [3:55:49, 615.19s/it]

---Run time is 14165.9980229 seconds ---

Accuracy of each fold: 
 [0.9090389016018307, 0.9533468559837728, 0.9949622166246851, 0.9286570743405276, 0.9958202716823407, 0.9464503042596348, 0.8239140588510042, 0.9972770592239619, 0.9437751004016064, 0.9735391400220507, 0.984573502722323, 0.9892318736539842, 0.744671201814059, 0.9554140127388535, 0.5628642797668609, 0.9871510855117412, 0.9890335846470185, 0.9803808424697057, 0.6935756551141167, 0.9785596481583287, 0.9961505560307955, 0.9968612680477087, 0.9957716701902748]

Avg accuracy : 
0.9270008766894428

Std of accuracy : 
0.11120923491905742
[0.9090389016018307, 0.9533468559837728, 0.9949622166246851, 0.9286570743405276, 0.9958202716823407, 0.9464503042596348, 0.8239140588510042, 0.9972770592239619, 0.9437751004016064, 0.9735391400220507, 0.984573502722323, 0.9892318736539842, 0.744671201814059, 0.9554140127388535, 0.5628642797668609, 0.9871510855117412, 0.9890335846470185, 0.9803808424697057, 0.6935756551141167, 0.9785596481583287,

In [9]:
crossvalidation = KFold(n_splits=23)
#for df in features_df_list:
df = features_maindf
acc_score = [];
Truth = [];
Output = [];
# n_estimators_list = []
import statistics
mode_estimators = statistics.mode(n_estimators_best)
#max_depth_list = []
for idx, subject in tqdm(enumerate(good_list)):
    
    train_index = ~df['Subject'].str.contains(subject)
    test_index = df['Subject'].str.contains(subject)
    X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
    y_train , y_test = df.loc[train_index,y_values].values.ravel(), df.loc[test_index,y_values]

    temp_model = AdaBoostClassifier(n_estimators=500, learning_rate = 0.1, algorithm="SAMME.R", random_state=2)
    param_distributions = {'n_estimators': randint(475, 526)}
    search = RandomizedSearchCV(estimator=temp_model, param_distributions=param_distributions, random_state=0)
    search.fit(X_train, y_train)

    n_estimators_list.append(search.best_params_['n_estimators'])
    #max_depth_list.append(search.best_params_['max_depth'])

    model = AdaBoostClassifier(n_estimators=mode_estimators, 
                               learning_rate=0.1,algorithm="SAMME.R", random_state=2)
    model.fit(X_train, y_train)    

    #model = AdaBoostClassifier(n_estimators=500, learning_rate = 0.1, algorithm="SAMME.R", random_state=2)
    #model.fit(X_train,y_train)

    pred_values = model.predict(X_test)
    pred_prob = model.predict_proba(X_test)[:, 1]
     
    acc = accuracy_score(y_test, pred_values)
    acc_score.append(acc)
    
    Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
    Output.extend(pred_values); ## it is a list
elapsed = timeit.default_timer() - start_time #gives total computation time
print("---Run time is %s seconds ---" % elapsed) #prints computation time
print()

print('Accuracy of each fold: \n {}'.format(acc_score))
print()
print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
print()
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print(acc_score)
print(confusion_matrix(Truth, Output))
print(classification_report(Truth, Output))
print(n_estimators_list)
# temp_list = []
# temp_idx = []
# n_estimators_best = []
# max_depth_best = []
# for acc in acc_score:
#     if acc >= 0.95:
#         temp_list.append(acc)
#         temp_idx.append(acc_score.index(acc))
# for idx in temp_idx:
#     n_estimators_best.append(n_estimators_list[idx])
#     max_depth_best.append(max_depth_list[idx])

23it [3:54:48, 612.54s/it]

---Run time is 28254.433449800003 seconds ---

Accuracy of each fold: 
 [0.9090389016018307, 0.9533468559837728, 0.9949622166246851, 0.9286570743405276, 0.9958202716823407, 0.9464503042596348, 0.8215787015413358, 0.9972770592239619, 0.9442771084337349, 0.9729878721058435, 0.984573502722323, 0.9892318736539842, 0.744671201814059, 0.9554140127388535, 0.5628642797668609, 0.9871510855117412, 0.9890335846470185, 0.9803808424697057, 0.6935756551141167, 0.9785596481583287, 0.9961505560307955, 0.9968612680477087, 0.9957716701902748]

Avg accuracy : 
0.9268971976810192

Std of accuracy : 
0.11129770957079084
[0.9090389016018307, 0.9533468559837728, 0.9949622166246851, 0.9286570743405276, 0.9958202716823407, 0.9464503042596348, 0.8215787015413358, 0.9972770592239619, 0.9442771084337349, 0.9729878721058435, 0.984573502722323, 0.9892318736539842, 0.744671201814059, 0.9554140127388535, 0.5628642797668609, 0.9871510855117412, 0.9890335846470185, 0.9803808424697057, 0.6935756551141167, 0.978559648158